In [1]:
import h5py
import cv2
import argparse
import os
import sys
import time
import zwoasi as asi
from time import time

from scipy import optimize
import laserbeamsize as lbs
import matplotlib.animation as animation

In [2]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.animation as animation

In [3]:
def save_dict_to_h5(ddict,filename='star_images.h5', initialdir="C:\\Starshot\\Polaris"):
    #use group with attributes to mock up dictionary, save to h5 file 'h5f'
    h5file=initialdir+'\\'+filename
    keys=ddict.keys()
    with h5py.File(h5file,'w') as h5f:
        for key in keys:
            h5f.create_dataset(key,data=np.array(ddict[key]))
        h5f.close()

In [4]:
def load_dict_from_h5(filename='star_images.h5', initialdir="C:\\Starshot\\Polaris"):
    #if there is a group called 'dictionsary', read its attributes as key/value, otherwise
    #read directly from an h5 file with no groups, datasets convert to key/value dictionary
    h5file=initialdir+'\\'+filename
    with h5py.File (h5file,'r') as h5f:
        h5keys=list(h5f.keys())
        if 'dictionary' in h5keys:
            grp=h5f.get('dictionary')
            out_dict = {}
            for key in grp.attrs.keys():
                out_dict[key]= grp.attrs[key]
        else:
            out_dict={}
            for key in h5keys:
                out_dict[key]=h5f[key][:]
        h5f.close()
    return out_dict

In [3]:
image

NameError: name 'image' is not defined

In [5]:
def star_centroid(image):
    #quick and dirty centroid, only viable for 1 star in field
    gray_image=image.copy()
    ret,thresh = cv2.threshold(gray_image,247,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    # calculate moments of binary image
    M = cv2.moments(thresh)
    # calculate x,y coordinate of center
    cX = (M["m10"] / M["m00"])
    cY = (M["m01"] / M["m00"])
    return cX,cY

In [6]:
#test=load_dict_from_h5(filename='polaris_1.h5')

In [7]:
env_filename=os.getenv('ZWO_ASI_LIB')
asi.init('C:\\Users\\peter\\Downloads\\ASI_Windows_SDK_V1.24\\ASI SDK\\lib\\x64\\ASICamera2.dll')

In [52]:
num_cameras = asi.get_num_cameras()
print(num_cameras,' Cameras')
camera=asi.Camera(0)
camera.set_control_value(asi.ASI_HIGH_SPEED_MODE, 1)
camera.set_roi()
if num_cameras==2:
    camera2=asi.Camera(1)
    camera2.set_control_value(asi.ASI_HIGH_SPEED_MODE, 1)
    camera2.set_roi() 

1  Cameras


In [9]:
asi.Camera.get_id??

In [46]:
camera.close()
#camera2.close()

In [11]:
%pylab auto
from time import time

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['time']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [20]:
pix_size=2.9  #microns/pixel
pix_scale = 206.6248*pix_size/750    #in arcsec/pixel
print(pix_scale)
micron_scale_remote = 206.6248/200  # in arcsec/micron_remote
remote_microns_per_local_pixel = pix_scale/micron_scale_remote
print('remote_microns_per_local_pixel',remote_microns_per_local_pixel)

0.7989492266666666
remote_microns_per_local_pixel 0.7733333333333333


In [21]:
print(pix_scale)

0.7989492266666666


In [22]:
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

In [69]:
def find_beacon_and_set_roi_max(camera=camera,n8s=8,exposure=10000,maxpix=None):
    camera.set_roi()
    camera.set_control_value(asi.ASI_EXPOSURE, exposure)
    test=camera.capture()
    figure()
    imshow(test)
    if maxpix==None:
        maxpix = np.unravel_index(np.argmax(test, axis=None), test.shape)
    start_x = int(maxpix[1]) - n8s*2
    start_y = int(maxpix[0]) - n8s*2
    print(maxpix)
    camera.set_roi(start_x=start_x,start_y=start_y,width=2*n8s*2,height=2*n8s*2)
    #test =  camera.capture()
    #figure(1)
    #imshow(test)
    return(start_x,start_y)

In [24]:
from scipy.signal import butter, filtfilt
import numpy as np

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

In [56]:
def vidcapture(camera=camera,nframes=100,exposure=3000,gain=500,n8s=8,maxpix=[500,500]):
    camera.set_control_value(asi.ASI_EXPOSURE, exposure)
    camera.set_control_value(asi.ASI_GAIN, gain)
    offsetxy = find_beacon_and_set_roi_max(camera=camera,exposure=10000,maxpix=maxpix,n8s=n8s)
    camera.set_control_value(asi.ASI_EXPOSURE, exposure)
    figure()
    imshow(camera.capture())
    print(offsetxy)
    ims=[]
    camera.start_video_capture()
    t0=time()
    x=[]
    y=[]
    for i in range(nframes):
        test=camera.capture_video_frame()
        xypos = star_centroid(test)
        x.append(xypos[0])
        y.append(xypos[1])
        ims.append(test) 
    camera.stop_video_capture()
    dt=time()-t0
    outdict={}
    outdict['xcen']=np.array(x)
    outdict['ycen']=np.array(y)
    outdict['images']=ims
    print('dt = ',dt)
    return outdict

In [81]:
tests=vidcapture(camera=camera,exposure=200,nframes=50000,n8s=4)

[500, 500]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


(492, 492)
dt =  11.305309772491455


In [83]:
figure()
imshow(tests['images'][0])

In [25]:
def vidcapture_star(camera=camera,nframes=100,exposure=3000,gain=500,arcsec_per_pix=0.7989492266666666):
    camera.set_control_value(asi.ASI_EXPOSURE, exposure)
    camera.set_control_value(asi.ASI_GAIN, gain)
    offsetxy = find_beacon_and_set_roi_max(camera=camera,exposure=10000)
    camera.set_control_value(asi.ASI_EXPOSURE, exposure)
    figure()
    imshow(camera.capture())
    print(offsetxy)
    ims=[]
    camera.start_video_capture()
    t0=time()
    x=[]
    y=[]
    for i in range(nframes):
        test=camera.capture_video_frame()
        xypos = star_centroid(test)
        x.append(xypos[0])
        y.append(xypos[1])
        ims.append(test) 
    camera.stop_video_capture()
    dt=time()-t0
    outdict={}
    outdict['xcen_arcsec']=np.array(x)*arcsec_per_pix
    outdict['ycen_arcsec']=np.array(y)*arcsec_per_pix
    outdict['images']=ims
    print('dt = ',dt)
    return outdict

In [26]:
def vidcapture_2_star(camera=camera,camera2=camera2,nframes=100,exposure=3000,gain=500,arcsec_per_pix=0.7989492266666666):
    camera.set_control_value(asi.ASI_EXPOSURE, exposure)
    camera.set_control_value(asi.ASI_GAIN, gain)
    camera2.set_control_value(asi.ASI_EXPOSURE, exposure)
    camera2.set_control_value(asi.ASI_GAIN, gain)
    offsetxy = find_beacon_and_set_roi_max(camera=camera,exposure=10000)
    camera.set_control_value(asi.ASI_EXPOSURE, exposure)
    offsetxy2 = find_beacon_and_set_roi_max(camera=camera2,exposure=10000)
    camera2.set_control_value(asi.ASI_EXPOSURE, exposure)

    figure()
    imshow(camera.capture())
    print(offsetxy)
    figure()
    imshow(camera2.capture())
    print(offsetxy2)

    ims=[]
    ims2=[]
    camera.start_video_capture()
    camera2.start_video_capture()
    t0=time()
    x=[]
    y=[]
    x2=[]
    y2=[]

    for i in range(nframes):
        test=camera.capture_video_frame()
        xypos = star_centroid(test)
        x.append(xypos[0])
        y.append(xypos[1])
        ims.append(test) 
        test2=camera2.capture_video_frame()
        xypos2 = star_centroid(test2)
        x2.append(xypos2[0])
        y2.append(xypos2[1])
        ims2.append(test2)
    camera.stop_video_capture()
    camera2.stop_video_capture()
    dt=time()-t0
    outdict={}
    outdict['xcen_arcsec']=np.array(x)*arcsec_per_pix
    outdict['ycen_arcsec']=np.array(y)*arcsec_per_pix
    outdict['images']=ims
    outdict['xcen2_arcsec']=np.array(x2)*arcsec_per_pix
    outdict['ycen2_arcsec']=np.array(y2)*arcsec_per_pix
    outdict['images2']=ims2

    print('dt = ',dt)
    return outdict

NameError: name 'camera2' is not defined

In [27]:
def vidcapture_sync(camera=camera,camera2=camera2,nframes=1000,exposure=1000,gain=350):
    camera.set_control_value(asi.ASI_EXPOSURE, exposure)
    camera.set_control_value(asi.ASI_GAIN, gain)
    camera2.set_control_value(asi.ASI_EXPOSURE, exposure)
    camera2.set_control_value(asi.ASI_GAIN, gain)
    camera.set_roi(start_x=500,start_y=500,width=128,height=128)
    camera2.set_roi(start_x=500,start_y=500,width=128,height=128)
    camera.start_video_capture()
    camera2.start_video_capture()
    t0=time()
    s1=[]
    s2=[]
    for i in range(nframes):
        test=camera.capture_video_frame()
        s1.append(sum(test))
        test2=camera2.capture_video_frame()
        s2.append(sum(test2))
    camera.stop_video_capture()
    camera2.stop_video_capture()
    dt=time()-t0
    print('dt = ',dt)
    return s1,s2

NameError: name 'camera2' is not defined

In [24]:
s1ssc,s2ssc=vidcapture_sync(camera=camera,camera2=camera2,exposure=10000,nframes=1000)

dt =  10.105336666107178


In [25]:
exposure=10
led_rate=10
figure()
title('Pulsed LED test of 2 camera synchronization- with test cables \n acquisitions at %s msec, LED %s Hz 50 duty cycle 1/10/24' %(str(exposure), str(led_rate)))
xlabel('Image sample')
ylabel('Sum of ROI pixels (mean subtracted) [arb]')
plot(s1ssc-mean(s1ssc),marker='x',label ='Camera 1')
plot(s2ssc-mean(s2ssc),marker='x',label = 'Camera 2')
legend()

In [56]:
exposure=2000
led_rate=10
figure()
title('Pulsed LED test of 2 camera synchronization \n acquisitions at %s msec, LED %s Hz duty cycle 1/10/24' %(str(exposure), str(led_rate)))
xlabel('Image sample')
ylabel('Sum of ROI pixels (mean subtracted) [arb]')
plot(s1-mean(s1),marker='x',label ='Camera 1')
plot(s2-mean(s2),marker='x',label = 'Camera 2')
legend()

In [152]:
camera.stop_video_capture()
camera2.stop_video_capture()

In [119]:
polaris_2x_13_240110_2ms=vidcapture_2_star(camera=camera,camera2=camera2,exposure=2000,nframes=50000)
save_dict_to_h5(polaris_2x_13_240110_2ms,filename='polaris_2x_13_240110_2ms_100sec_capture.h5')

(649, 1653)
(994, 1503)
(1589, 585)
(1439, 930)
dt =  100.53560972213745


In [83]:
ddict=polaris_2x_5_240110_2ms.copy()
figure(figsize=[8,6])
xlabel('X position [arcsec]')
ylabel('Y position [arcsec]')
xsdev=np.std(ddict['xcen_arcsec'])
ysdev=np.std(ddict['ycen_arcsec'])

xsdev2=np.std(ddict['xcen2_arcsec'])
ysdev2=np.std(ddict['ycen2_arcsec'])

title('Polaris 2 camera positions at 500 Hz, 1/10/24(5), 100sec.  Upgraded centroids:\n  xstdev:'+str(xsdev)+' ystdev:'+str(ysdev)+ '\n x2stdev:'+str(xsdev2)+' y2stdev:'+str(ysdev2))
plot(ddict['xcen_arcsec'],ddict['ycen_arcsec'],'x',label='Camera 1')
plot(ddict['xcen2_arcsec'],ddict['ycen2_arcsec'],'x',label='Camera 2')
legend()
savefig('Polaris/polaris_2x_5_240110_2ms_100sec_xy.png')

In [112]:
ddict=polaris_2x_11_240110_2ms.copy()
figure(figsize=[8,6])
xlabel('Sample [2ms]')
ylabel('X position [arcsec]')
xsdev=np.std(ddict['xcen_arcsec'])
ysdev=np.std(ddict['ycen_arcsec'])

xsdev2=np.std(ddict['xcen2_arcsec'])
ysdev2=np.std(ddict['ycen2_arcsec'])

title('Polaris 2 camera positions at 500 Hz, 1/10/24(11), 100sec\nUpgraded centroids:  xstdev:'+str(xsdev)+' ystdev:'+str(ysdev))
plot(ddict['xcen_arcsec'],label='Camera 1')
plot(ddict['xcen2_arcsec'],label='Camera 2')
legend()
savefig('Polaris/polaris_2x_11_240110_2ms_100sec_xx.png')

In [124]:
ddict=polaris_2x_11_240110_2ms.copy()
figure(figsize=[8,6])
xlabel('Sample [3ms]')
ylabel('Y position [arcsec]')
xsdev=np.std(ddict['xcen_arcsec'])
ysdev=np.std(ddict['ycen_arcsec'])

xsdev2=np.std(ddict['xcen2_arcsec'])
ysdev2=np.std(ddict['ycen2_arcsec'])

title('Polaris 2 camera positions at 500 Hz, 1/10/24(11), 100sec')
plot(ddict['ycen_arcsec'],label='Camera 1')
plot(ddict['ycen2_arcsec'],label='Camera 2')
legend()
savefig('Polaris/polaris_2x_11_240110_2ms_100sec_yy.png')

In [142]:
nsmth=10 #smoothing bin width
hp=1  #highpass cutoff in hz
fs=500  #sample rate in hz
ddict=polaris_2x_10_240110_2ms.copy()
figure(figsize=[8,6])
xlabel('Sample [3ms]')
#ylabel('Y position [arcsec]')
#xlabel('Cam 1 X position [arcsec]')
ylabel('Cam 1, 2 Y position [arcsec]')

x1 = smooth(butter_highpass_filter(ddict['ycen_arcsec'],hp,fs),box_pts=nsmth)
x2 = smooth(butter_highpass_filter(ddict['ycen2_arcsec'],hp,fs),box_pts=nsmth)
title('Polaris 2 camera positions at 500 Hz, 1/10/24(10), 100sec \n Highpass %s Hz smoothed %s moving average' %(str(hp), str(nsmth)))
#plot(x1,x2,'x',label='Camera 1')
plot(x1,label='Camera 1')
plot(x2,label='Camera 2')
legend()
#savefig('Polaris/polaris_2x_11_240110_2ms_100sec_yy.png')

In [123]:
ddict=polaris_240109_8_2ms.copy()
figure(figsize=[8,6])
xlabel('X position [arcsec]')
ylabel('Y position [arcsec]')
xsdev=np.std(ddict['xcen_arcsec'])
ysdev=np.std(ddict['ycen_arcsec'])
title('Polaris_8 position at 500 Hz, 1/9/24, 100sec\nUpgraded centroids:  xstdev:'+str(xsdev)+' ystdev:'+str(ysdev))
plot(ddict['xcen_arcsec'],ddict['ycen_arcsec'],'x')
#savefig('Polaris/polaris_240109_8_2ms_100sec_xy.png')

NameError: name 'polaris_240109_8_2ms' is not defined

In [91]:
figure()
imshow(polaris_2x_5_240110_2ms['images2'][100])

In [44]:
camera.stop_video_capture()
camera2.stop_video_capture()

In [173]:
arcsec_per_pix=0.7989492266666666
print(polaris_240109_4_5ms['xcen_arcsec'][0]/arcsec_per_pix)
print(polaris_240109_4_5ms['ycen_arcsec'][0]/arcsec_per_pix)
figure()
imshow(polaris_240109_4_5ms['images'][0])

64.85607709750568
67.56395691609977


In [224]:
def replace_xy(in_dict,arcsec_per_pix=0.7989492266666666):
    #use star_centroid to replace xy estimate with centroid one (better?)
    out_dict={}
    out_dict['images'] = [] 
    out_dict['xcen_arcsec'] = []
    out_dict['ycen_arcsec'] = []
    
    for img in in_dict['images']:
        out_dict['images'].append(img)
        xcen,ycen = star_centroid(img)
        out_dict['xcen_arcsec'].append(xcen*arcsec_per_pix)
        out_dict['ycen_arcsec'].append(ycen*arcsec_per_pix)
    return out_dict

In [231]:
polaris_240109_1_5ms_r=replace_xy(polaris_240109_1_5ms)
polaris_240109_2_1ms_r=replace_xy(polaris_240109_2_1ms)
polaris_240109_3_1ms_r=replace_xy(polaris_240109_3_1ms)
polaris_240109_4_5ms_r=replace_xy(polaris_240109_4_5ms)


In [227]:
len(polaris_240109_1_5ms_r['xcen_arcsec'])

2000

In [222]:
gray_image = polaris_240109_1_5ms['images'][1].copy()
ret,thresh = cv2.threshold(gray_image,127,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
figure()
imshow(thresh)


In [268]:
img = polaris_240109_5_2ms['images'] # some array of images
frames = [] # for storing the generated images
fig = plt.figure()
for i in range(len(img)):
    frames.append([plt.imshow(img[i], cmap=cm.Greys_r,animated=True)])

ani = animation.ArtistAnimation(fig, frames, interval=50, blit=True,
                                repeat_delay=1000)
ani.save('Polaris_240109_5_500Hz_10sec.mp4')


In [53]:
# Create two subplots and unpack the output array immediately
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
ax1.plot(x, y)
ax1.set_title('Sharing Y axis')
ax2.scatter(x, y)

AttributeError: 'tuple' object has no attribute 'flatten'

In [146]:
img1 = polaris_2x_10_240110_2ms['images'][:500] # some array of images
img2 = polaris_2x_10_240110_2ms['images2'][:500] # some array of images
frames = [] # for storing the generated images
fig,(ax1,ax2) = plt.subplots(1,2)
for i in range(len(img1)):
    frames.append([ax1.imshow(img1[i], cmap=cm.Greys_r,animated=True),ax2.imshow(img2[i], cmap=cm.Greys_r,animated=True)])

ani = animation.ArtistAnimation(fig, frames, interval=50, blit=True,
                                repeat_delay=1000)
ani.save('Polaris_11_2x.mp4')


In [132]:
figure()
imshow(ddict['images'][1300])

In [41]:
x1,y1,ims1,x2,y2,ims=vidcapture2(camera1=camera,camera2=camera2,nframes=10000,offsetxy1=[512,512],n8s=8)

dt =  11.810209035873413


In [8]:
def vidcapture2(camera1=camera,camera2=camera2,nframes=10,exposure1=32,gain1=400,exposure2=32,gain2=0,offsetxy1=None,n8s=8):
    camera1.set_control_value(asi.ASI_EXPOSURE, exposure1)
    camera1.set_control_value(asi.ASI_GAIN, gain1)
    if offsetxy1 == None:
        offsetxy1 = find_beacon_and_set_roi_max(camera=camera1,n8s=n8s)
    else:
        start_x = offsetxy1[0] - 4*n8s*8
        start_y = offsetxy1[1] - n8s*8
        camera1.set_roi(start_x=start_x,start_y=start_y,width=2*n8s*8,height=2*n8s*8)
        camera2.set_roi(start_x=start_x,start_y=start_y,width=2*n8s*8,height=2*n8s*8)
    camera2.set_control_value(asi.ASI_EXPOSURE, exposure2)
    camera2.set_control_value(asi.ASI_GAIN, gain2)
    offsetxy2 = offsetxy1  #find_beacon_and_set_roi_max(camera=camera2)

    ims1=[]
    ims2=[]
    camera1.start_video_capture()
    camera2.start_video_capture()
    t0=time()
    x1=[]
    y1=[]
    x2=[]
    y2=[]
    for i in range(nframes):
        test1=camera1.capture_video_frame()
        beamparam1 = lbs.basic_beam_size(test1)
        x1.append(beamparam1[0])
        y1.append(beamparam1[1])
        ims1.append(test1) 
        test2=camera2.capture_video_frame()
        beamparam2 = lbs.basic_beam_size(test2)
        x2.append(beamparam2[0])
        y2.append(beamparam2[1])
        ims2.append(test2) 
    camera1.stop_video_capture()
    camera2.stop_video_capture()
    dt=time()-t0
    print('dt = ',dt)
    return x1,y1,ims1,x2,y2,ims2

NameError: name 'camera2' is not defined

In [38]:
camera2.stop_video_capture()

In [47]:
freqs1=[]
xamplitudes1=[]
yamplitudes1=[]
for freq in linspace(1,500,100):
    freq=int(freq)
    print('Frequency',freq)
    set_freq_volts(sdg1025,freq,2.0)
    x,y,ims = vidcapture(camera,nframes=1000,exposure=32,gain=0)
    xamp = max(x)-min(x)
    yamp = max(y)-min(y)
    xamplitudes1.append(xamp)
    yamplitudes1.append(yamp)
    freqs1.append(freq)

In [54]:
figure()
title('Beacon fiber motion from image at ~1kHz framerate \n 750mm/200mm Fl telescope, 4V PP on FTA')
xlabel('Frequency [Hz]')
ylabel('X p-p amplitude [$\mu$]')
legend()
plot(freqs1,np.array(xamplitudes1)*remote_microns_per_local_pixel)

No handles with labels found to put in legend.


In [10]:
%pylab auto
from time import time

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['time']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [51]:
figure(2)
camera.set_roi()
camera.set_control_value(asi.ASI_EXPOSURE, 1000)
camera.set_control_value(asi.ASI_GAIN, 300)
test=camera.capture()
imshow(test)

ZWO_IOError: Camera closed

In [13]:
figure(3)
camera2.set_roi()
camera2.set_control_value(asi.ASI_EXPOSURE, 10000)
camera2.set_control_value(asi.ASI_GAIN, 550)
test=camera2.capture()
imshow(test)


NameError: name 'camera2' is not defined